In [1]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 4.4MB/s 
     |████████████████████████████████| 71kB 11.2MB/s 
     |████████████████████████████████| 81kB 8.5MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=621567 sha256=dead0aebe2d6d72ec3940f96016a27d85e96606eee95a6e11ecaaf4da1b5e5e9
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=513ddcfcc750e3999383cc9f382c021a8505f1d9494694ac473138cf4a18f401
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools


In [ ]:
import numpy as np
from pycuda import compiler, gpuarray, tools
import pycuda.driver as drv
import pycuda.autoinit

In [70]:
MATRIX_SIZES = [128, 256, 512, 1024,2048]
BLOCK_SIZE = 16

In [36]:
kernel_code_template = """
__global__ void matrix_multiply(int matrixsize,float *a, float *b, float *c)
{
    // 2D Thread ID 
    int tx = blockDim.x*blockIdx.x + threadIdx.x; // Compute column index
    int ty = blockDim.y*blockIdx.y + threadIdx.y; // Compute row index
    // Each thread loads one row of M and one column of N, 
    //   to produce one element of P.
    if((ty <matrixsize) && (tx < matrixsize))
    {
    // Pvalue is used to store the element of the matrix
    // that is computed by the thread
    float Pvalue = 0;
    for(int k=0; k<matrixsize; ++k)
    {
    float Aelement = a[ty*matrixsize +k];
    float Belement = b[k*matrixsize +tx];
    Pvalue += Aelement * Belement;
    }
    c[ty * matrixsize + tx] = Pvalue;
    }
}
"""

# compile the kernel code
mod = compiler.SourceModule(kernel_code_template)

# get the kernel function from the compiled module
matrix_multiply = mod.get_function("matrix_multiply")

In [60]:
def multiply_with_cpu(a, b):
  return a.dot(b)

In [61]:
def multiply_with_gpu(a, b, MATRIX_SIZE):
  # transfer host (CPU) memory to device (GPU) memory
  a_gpu = gpuarray.to_gpu(a)
  b_gpu = gpuarray.to_gpu(b)

  # create empty gpu array for the result (C = A * B)
  c_gpu = gpuarray.empty((MATRIX_SIZE, MATRIX_SIZE), np.float32)
  # set grid size
  #if MATRIX_SIZE%BLOCK_SIZE != 0:
    #  grid=(MATRIX_SIZE//BLOCK_SIZE+1,MATRIX_SIZE//BLOCK_SIZE+1,1)
  #else:
  grid=(MATRIX_SIZE//BLOCK_SIZE,MATRIX_SIZE//BLOCK_SIZE,1)

  # call the kernel on the card
  matrix_multiply(np.uint32(MATRIX_SIZE),
    # inputs
    a_gpu, b_gpu,
    # output
    c_gpu,
    grid=grid,
    block = (BLOCK_SIZE, BLOCK_SIZE, 1),
    )
  return c_gpu  

In [62]:
def calculate(a, b, MATRIX_SIZE):
  start_cpu = timer()
  c_cpu = multiply_with_cpu(a, b)
  cpu_multiply_time = timer() - start_cpu

  start_gpu = timer()
  c_gpu = multiply_with_gpu(a, b, MATRIX_SIZE)
  gpu_multiply_time = timer() - start_gpu
  
  return cpu_multiply_time * 1000, gpu_multiply_time * 1000, np.allclose(c_cpu, c_gpu.get())

In [71]:
count = 15

print(" N \t CPU time \t GPU time \t Speedup")

for size in MATRIX_SIZES:
  cpu_time = 0
  gpu_time = 0

  for i in range(count):
    a = np.random.rand(size, size).astype(np.float32)
    b = np.random.rand(size, size).astype(np.float32)

    current_cpu_time, current_gpu_time, err = calculate(a, b, size)
    cpu_time += current_cpu_time
    gpu_time += current_gpu_time

    if err is False:
      print("N = {:d}: results not equals".format(size))

  print("{:4d} \t {:7.3f} \t {:7.3f} \t {:7.2f}".format(size, cpu_time / count, gpu_time / count, cpu_time / gpu_time))

 N 	 CPU time 	 GPU time 	 Speedup
 128 	   0.122 	   0.422 	    0.29
 256 	   0.613 	   0.615 	    1.00
 512 	   4.498 	   2.164 	    2.08
1024 	  33.361 	   9.894 	    3.37
2048 	 258.712 	  62.114 	    4.17
